# Banxico Parte 2:

* Ocupando **únicamente** pandas, numpy; matplotlib; seaborn, con la información obtenida en la parte 1 (pandas DataFrame) modelar con Naïve Bayes si un documento dado pertence a la clase `mantiene`, `sube`, `baja`

## Pasos
1. Descargar PDFs
2. Limpiar los datos
3. Eliminar de todos los archivo las palabras que contengan `mantiene`, `sube`, `baja`.
4. Parte los documentos entre un set de entrenamiento (0.8) y uno de prueba (0.2)
5. Entrena el modelo con los documentos de entrenamiento y valida el resultado con una matriz de confusión usado la base de prueba

In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from collections import defaultdict
import PyPDF2
from sklearn.model_selection import train_test_split
import re
import pickle
import matplotlib.pyplot as plt
import tqdm
from unidecode import unidecode
from nltk import corpus
from collections import Counter


In [2]:
df = pd.read_pickle("comunicados-banxico-final.pkl")
df.head()

,text,url,category
date,,,
2020-02-13,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,disminuye
2019-12-19,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,disminuye
2019-11-14,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,disminuye
2019-09-26,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,disminuye
2019-08-15,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,disminuye


In [3]:
#Descargamos  los comunicados (ya habíamos probado en la primera parte de la tarea)
for i in range(len(df)):
    url = df['url'][i]
    r = requests.get(url)
    
    paper = r.content

    # recomendable primero crear una carpeta si no existe donde guardaré mis pdfs
    if not os.path.exists("comunicados_pdfs"):  
        os.mkdir("comunicados_pdfs")
    
    full_path = os.path.join("comunicados_pdfs", f"paper{i}.pdf")
    
    # escribe cada pdf en mi carpeta
    with open(full_path,"wb") as f: # wb write binary
        f.write(paper)

In [4]:
#Leemos los papers y los guardamos en un diccionario 
d = defaultdict(list)

for i in range(len(df)):
    
    full_path = os.path.join("comunicados_pdfs", f"paper{i}.pdf")
    
    pdf_file = open(full_path, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    c = Counter(range(number_of_pages))

    text_paper_i = ''
    for j in c:
        page = read_pdf.getPage(j)
        page_content = page.extractText()
        text_paper_i = text_paper_i + page_content # leemos el pdf completo y lo guardamos en text_paper_i
            
    d[df['category'][i]].append(text_paper_i)

In [5]:
#utilizamos como llaves de muestro diccionario las categorias que  se asignaron en la primera parte del proyecto 
k1, k2, k3 = 'mantiene', 'incrementa', 'disminuye'

In [6]:
#Definimos una función que nos ayuda a quitar cualquier caracter especial 
def clean_text(texto):
    cln = texto.lower()
    cln = re.sub("[^\w ]", "", cln) 
    return cln


In [7]:

D1 = [clean_text(doc) for doc in d[k1]]
D2 = [clean_text(doc) for doc in d[k2]]
D3 = [clean_text(doc) for doc in d[k3]]

# quiero partir mis datos en un train group y un test group de datos

D1, D1_test = train_test_split(D1,test_size = 0.2)
D2, D2_test = train_test_split(D2,test_size = 0.2)
D3, D3_test = train_test_split(D3,test_size = 0.2)

In [8]:
N1 = len(D1)
N2 = len(D2)
N3 = len(D3)
N = N1 + N2 + N3

In [9]:
#continuamos limpiando el texto, quitando las stopwords, y las palabras que no nos intersan  
stopwords = corpus.stopwords.words("spanish")
key_related_stopwords = ['mantener','disminuir','incrementar', 'mantengan','disminuyan','incrementen']
stopwords = stopwords + key_related_stopwords

In [10]:

V1 = [word for txt in D1 for word in txt.split() if word not in stopwords]
V2 = [word for txt in D2 for word in txt.split() if word not in stopwords]
V3 = [word for txt in D3 for word in txt.split() if word not in stopwords]

In [11]:
V1_c = pd.Series(V1, name='k1').value_counts()
V2_c = pd.Series(V2, name = 'k2').value_counts()
V3_c = pd.Series(V3, name = 'k3').value_counts()

In [12]:
V = set(V1) | set(V2) | set(V3)
v = pd.DataFrame(index=V)

In [14]:
L = (v.join(V1_c, how = "left")
    .join(V2_c, how = "left")
    .join(V3_c, how = "left"))


In [15]:
#Funcion de verosimilitud

L = L.fillna(0) + 1  
L = L / L.sum(axis=0) 

L.sort_values("k1",ascending = False)
L.sample(10)

,k1,k2,k3
divergente,0.000061,0.000158,0.000125
implementan,0.000061,0.000079,0.000125
existían,0.000061,0.000079,0.000125
1198,0.000061,0.000079,0.000125
especial,0.000637,0.000789,0.000249
utilizan,0.000091,0.000237,0.000125
criterios,0.000061,0.000079,0.000125
substancial,0.000030,0.000079,0.000249
vías,0.000061,0.000079,0.000125
contraído,0.000030,0.000079,0.000249


In [16]:
#Funcion de log-verosimilitud
log_L = np.log(L)

log_prior_k1 = np.log(N1/N)
log_prior_k2 = np.log(N2/N)
log_prior_k3 = np.log(N3/N)

log_priors = np.array([log_prior_k1,log_prior_k2,log_prior_k3])

In [17]:
len(D1_test)
d_test = D1_test[0]
f = set([wrd for wrd in d_test.split() if wrd in V])

In [18]:
res = log_L.loc[f].sum(axis=0) + log_priors
res

k1   -2252.002225
k2   -2377.143994
k3   -2359.301575
dtype: float64

In [19]:
res.idxmax()

'k1'

In [20]:
D_test = D1_test + D2_test + D3_test
print(len(D_test))
y_test = ["k1"]*len(D1_test) + ["k2"]*len(D2_test) + ["k3"]*len(D3_test)

23


In [21]:

yhat_test = [] 
for d_test in D_test:
    f = set([palabra for palabra in d_test.split() if palabra in V])
    res = log_L.loc[f].sum(axis=0) + log_priors
    yhat_test.append(res.idxmax())

print(y_test)
print(yhat_test)

['k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k1', 'k2', 'k2', 'k2', 'k2', 'k3', 'k3', 'k3', 'k3']
['k1', 'k1', 'k1', 'k1', 'k3', 'k2', 'k1', 'k1', 'k1', 'k3', 'k3', 'k1', 'k1', 'k1', 'k1', 'k2', 'k2', 'k2', 'k1', 'k2', 'k2', 'k2', 'k3']


In [22]:
from  sklearn.metrics import confusion_matrix
confusion_matrix(y_test,yhat_test)

array([[11,  1,  3],
       [ 1,  3,  0],
       [ 0,  3,  1]], dtype=int64)